In [20]:
from pathlib import Path
import sys  
import os
import pandas as pd 
import scipy.sparse as sps
from datetime import datetime

import numpy as np
from operator import itemgetter
import optuna as op

In [11]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [14]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

In [25]:
print(f"Train size: {URM_train.getnnz()},  Val size: {URM_val.getnnz()}, Test size: {URM_test.getnnz()}")

Train size: 1274929,  Val size: 224987, Test size: 264691


In [15]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [16]:
from libs.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

In [26]:
def objective_function_PureSVD(optuna_trial):
    recommender_instance = PureSVDRecommender(URM_train)
    recommender_instance.fit(
        num_factors = optuna_trial.suggest_int('num_factors', 10, 1000, log=True),
    )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)  
    
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["MAP"]

In [27]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [28]:
optuna_study_PureSVD = op.create_study(direction="maximize")
        
save_results_PureSVD = SaveResults("result_experiments/PureSVD/results_optuna.csv")
        
optuna_study_PureSVD.optimize(objective_function_PureSVD,
                      callbacks=[save_results_PureSVD],
                      n_trials = 400)

[I 2025-01-06 21:51:33,109] A new study created in memory with name: no-name-7e871fed-8fc1-4e5f-9c55-e3c539743857


PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.86 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 18.51 sec. Users per second: 1876


[I 2025-01-06 21:51:52,508] Trial 0 finished with value: 0.010978252075997678 and parameters: {'num_factors': 85}. Best is trial 0 with value: 0.010978252075997678.


PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 4.11 sec


[W 2025-01-06 21:52:16,743] Trial 1 failed with parameters: {'num_factors': 374} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/tr/d1rx7zt15lzgkp2yc1nwbg100000gn/T/ipykernel_68669/4168386530.py", line 7, in objective_function_PureSVD
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/safuan/Python/RecSy

KeyboardInterrupt: 